## CBoW മോഡൽ പരിശീലനം

ഈ നോട്ട്‌ബുക്ക് [AI for Beginners Curriculum](http://aka.ms/ai-beginners) എന്ന കോഴ്സിന്റെ ഭാഗമാണ്

ഈ ഉദാഹരണത്തിൽ, നാം CBoW ഭാഷാ മോഡൽ പരിശീലിപ്പിച്ച് നമ്മുടെ സ്വന്തം Word2Vec എംബെഡ്ഡിംഗ് സ്പേസ് ഉണ്ടാക്കുന്നത് കാണാം. ടെക്സ്റ്റിന്റെ ഉറവിടമായി AG News ഡാറ്റാസെറ്റ് ഉപയോഗിക്കും.


In [30]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

നാം ഡാറ്റാസെറ്റ് ലോഡ് ചെയ്യുന്നതിലൂടെ ആരംഭിക്കും:


In [1]:
ds_train, ds_test = tfds.load('ag_news_subset').values()

## CBoW മോഡൽ

CBoW ഒരു വാക്ക് പ്രവചിക്കാൻ അതിന്റെ $2N$ സമീപവർത്തി വാക്കുകൾ അടിസ്ഥാനമാക്കി പഠിക്കുന്നു. ഉദാഹരണത്തിന്, $N=1$ ആണെങ്കിൽ, *I like to train networks* എന്ന വാചകത്തിൽ നിന്നുള്ള ജോഡികൾ ഇങ്ങനെ ആയിരിക്കും: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). ഇവിടെ, ആദ്യ വാക്ക് ഇൻപുട്ടായി ഉപയോഗിക്കുന്ന സമീപവർത്തി വാക്കാണ്, രണ്ടാമത്തെ വാക്ക് പ്രവചിക്കേണ്ട വാക്കാണ്.

അടുത്ത വാക്ക് പ്രവചിക്കാൻ ഒരു നെറ്റ്‌വർക്ക് നിർമ്മിക്കാൻ, സമീപവർത്തി വാക്ക് ഇൻപുട്ടായി നൽകുകയും വാക്കിന്റെ നമ്പർ ഔട്ട്പുട്ടായി ലഭിക്കുകയും വേണം. CBoW നെറ്റ്‌വർക്കിന്റെ ഘടന ഇപ്രകാരം ആണ്:

* ഇൻപുട്ട് വാക്ക് embedding ലെയറിലൂടെ കടന്നുപോകും. ഈ embedding ലെയർ തന്നെ നമ്മുടെ Word2Vec embedding ആയിരിക്കും, അതിനാൽ ഞങ്ങൾ അതിനെ `embedder` എന്ന വ്യത്യസ്ത വേരിയബിളായി നിർവചിക്കും. ഈ ഉദാഹരണത്തിൽ embedding വലുപ്പം 30 ആക്കി ഉപയോഗിക്കും, എന്നാൽ നിങ്ങൾക്ക് ഉയർന്ന ഡൈമെൻഷനുകളുമായി പരീക്ഷിക്കാൻ ആഗ്രഹമുണ്ടാകാം (യഥാർത്ഥ word2vec 300 ആണ്)
* embedding വെക്ടർ പിന്നീട് ഒരു ഡെൻസ് ലെയറിലേക്ക് കടന്നുപോകും, അത് ഔട്ട്പുട്ട് വാക്ക് പ്രവചിക്കും. അതിനാൽ ഇതിൽ `vocab_size` ന്യുറോണുകൾ ഉണ്ടാകും.

Keras-യിലെ embedding ലെയർ സംഖ്യാത്മക ഇൻപുട്ട് വാക്കിനെ ഓൺ-ഹോട്ട് എൻകോഡിങ്ങിലേക്ക് എങ്ങനെ മാറ്റണമെന്ന് സ്വയം അറിയുന്നു, അതിനാൽ ഇൻപുട്ട് വാക്ക് പ്രത്യേകം ഓൺ-ഹോട്ട് എൻകോഡ് ചെയ്യേണ്ടതില്ല. `input_length=1` എന്ന് നമുക്ക് വ്യക്തമാക്കുന്നു, ഇൻപുട്ട് സീക്വൻസിൽ ഒരു വാക്ക് മാത്രം വേണമെന്ന് — സാധാരണയായി embedding ലെയർ ദീർഘമായ സീക്വൻസുകളുമായി പ്രവർത്തിക്കാൻ രൂപകൽപ്പന ചെയ്തതാണ്.

ഔട്ട്പുട്ടിനായി, നാം `sparse_categorical_crossentropy` നഷ്ട ഫംഗ്ഷൻ ഉപയോഗിച്ചാൽ, പ്രതീക്ഷിക്കുന്ന ഫലമായി വാക്കിന്റെ നമ്പറുകൾ മാത്രം നൽകണം, ഓൺ-ഹോട്ട് എൻകോഡിങ്ങ് വേണ്ട.

കണക്കുകൾ കുറയ്ക്കാൻ `vocab_size` 5000 ആയി നിശ്ചയിക്കും. പിന്നീട് ഉപയോഗിക്കാൻ ഒരു വെക്ടറൈസറും നിർവചിക്കും.


In [68]:
vocab_size = 5000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))
embedder = keras.layers.Embedding(vocab_size,30,input_length=1)

model = keras.Sequential([
    embedder,
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1, 30)             150000    
                                                                 
 dense_1 (Dense)             (None, 1, 5000)           155000    
                                                                 
Total params: 305,000
Trainable params: 305,000
Non-trainable params: 0
_________________________________________________________________


വെക്ടറൈസർ ആരംഭിച്ച് വാക്കുകളുടെ ശബ്ദകോശം പുറത്തെടുക്കാം:


In [69]:
def extract_text(x):
    return x['title']+' '+x['description']

vectorizer.adapt(ds_train.take(500).map(extract_text))
vocab = vectorizer.get_vocabulary()

## പരിശീലന ഡാറ്റ തയ്യാറാക്കൽ

ഇപ്പോൾ നാം ടെക്സ്റ്റിൽ നിന്ന് CBoW വാക്ക് ജോഡികൾ കണക്കാക്കുന്ന പ്രധാന ഫംഗ്ഷൻ പ്രോഗ്രാം ചെയ്യാം. ഈ ഫംഗ്ഷൻ വിൻഡോ സൈസ് നിർദ്ദേശിക്കാൻ അനുവദിക്കും, കൂടാതെ ഇൻപുട്ട്-ഔട്ട്പുട്ട് വാക്കുകളുടെ ഒരു സെറ്റ് തിരികെ നൽകും. ഈ ഫംഗ്ഷൻ വാക്കുകളിലും വെക്ടറുകളിലും/ടെൻസറുകളിലും ഉപയോഗിക്കാവുന്നതാണ് - ഇത് ടെക്സ്റ്റ് എൻകോഡ് ചെയ്ത് `to_cbow` ഫംഗ്ഷനിലേക്ക് അയയ്ക്കാൻ സഹായിക്കും.


In [70]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(vectorizer('I like to train networks')))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=3>]

പരിശീലന ഡാറ്റാസെറ്റ് തയ്യാറാക്കാം. എല്ലാ വാർത്തകളും പരിശോധിച്ച്, വാക്കുകളുടെ ജോഡികളുടെ പട്ടിക ലഭിക്കാൻ `to_cbow` വിളിക്കാം, ആ ജോഡികൾ `X`ക്കും `Y`ക്കും ചേർക്കാം. സമയപരിധി കണക്കിലെടുത്ത്, ആദ്യം 10,000 വാർത്തകൾ മാത്രം പരിഗണിക്കും - നിങ്ങൾക്ക് കൂടുതൽ സമയം ഉണ്ടെങ്കിൽ ഈ പരിധി എളുപ്പത്തിൽ നീക്കം ചെയ്യാം, മികച്ച എംബെഡിംഗുകൾ നേടാൻ :)


In [100]:
X = []
Y = []
for i,x in zip(range(10000),ds_train.map(extract_text).as_numpy_iterator()):
    for w1, w2 in to_cbow(vectorizer(x),window_size=1):
        X.append(tf.expand_dims(w1,0))
        Y.append(tf.expand_dims(w2,0))

അത് ഡാറ്റയും ഒരു ഡാറ്റാസെറ്റായി മാറ്റുകയും പരിശീലനത്തിനായി ബാച്ച് ചെയ്യുകയും ചെയ്യും:


In [101]:
ds = tf.data.Dataset.from_tensor_slices((X,Y)).batch(256)

ഇപ്പോൾ നമുക്ക് യഥാർത്ഥ പരിശീലനം നടത്താം. നാം വളരെ ഉയർന്ന ലേണിംഗ് റേറ്റുള്ള `SGD` ഓപ്റ്റിമൈസർ ഉപയോഗിക്കും. നിങ്ങൾക്ക് `Adam` പോലുള്ള മറ്റ് ഓപ്റ്റിമൈസറുകളുമായി പരീക്ഷണം നടത്താനും കഴിയും. ആദ്യം നാം 200 എപ്പോക്കുകൾക്ക് പരിശീലനം നടത്തും - കുറവ് നഷ്ടം നേടാൻ നിങ്ങൾക്ക് ഈ സെൽ വീണ്ടും റൺ ചെയ്യാം.


In [102]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.1),loss='sparse_categorical_crossentropy')
model.fit(ds,epochs=200)

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


2156/2156 [==============================] - 7s 3ms/step - loss: 5.6134
Epoch 2/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5431
Epoch 3/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5029
Epoch 4/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4754
Epoch 5/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4548
Epoch 6/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4382
Epoch 7/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4243
Epoch 8/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4123
Epoch 9/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4019
Epoch 10/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3926
Epoch 11/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3843
Epoch 12/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3767
E

## Word2Vec പരീക്ഷിക്കുന്നു

Word2Vec ഉപയോഗിക്കാൻ, നമുക്ക് നമ്മുടെ വാക്ക് സമാഹാരത്തിലെ എല്ലാ വാക്കുകൾക്കും അനുയോജ്യമായ വെക്ടറുകൾ എടുക്കാം:


In [103]:
vectors = embedder(vectorizer(vocab))
vectors = tf.reshape(vectors,(-1,30)) # we need reshape to get rid of extra dimension

ഉദാഹരണത്തിന്, **Paris** എന്ന വാക്ക് എങ്ങനെ ഒരു വെക്ടറിലേക്ക് എൻകോഡ് ചെയ്യപ്പെടുന്നു എന്ന് നോക്കാം:


In [104]:
paris_vec = embedder(vectorizer('paris'))[0]
print(paris_vec)

tf.Tensor(
[-0.13308628  0.50972325  0.00344684  0.185389   -0.03176536  0.22262476
 -0.3856765  -0.6854793   0.5185803  -0.7215402  -0.16101503  0.15622072
  0.00653811 -0.14954254  0.03379822 -0.01243829  0.27907634 -0.32538188
  0.21718933  0.31112966 -0.24142407  0.15589055  0.2915561   0.19029242
  0.08425518 -0.0941902  -0.54313695 -0.24854654  0.26196313  0.18027727], shape=(30,), dtype=float32)


സമാനാർത്ഥകങ്ങൾ കണ്ടെത്താൻ Word2Vec ഉപയോഗിക്കുന്നത് രസകരമാണ്. നൽകിയിരിക്കുന്ന ഫംഗ്ഷൻ ഒരു നൽകിയ ഇൻപുട്ടിനോട് ഏറ്റവും അടുത്ത `n` വാക്കുകൾ തിരികെ നൽകും. അവ കണ്ടെത്താൻ, $|w_i - v|$ എന്ന നോർം കണക്കാക്കുന്നു, ഇവിടെ $v$ നമ്മുടെ ഇൻപുട്ട് വാക്കിന്റെ വെക്ടറാണ്, $w_i$ വാക്ക് കോശത്തിലെ i-ആം വാക്കിന്റെ എൻകോഡിങ്ങാണ്. തുടർന്ന് ആrray സോർട്ട് ചെയ്ത് `argsort` ഉപയോഗിച്ച് അനുയോജ്യമായ ഇൻഡക്സുകൾ തിരികെ നൽകുന്നു, പിന്നീട് വാക്ക് കോശത്തിലെ ഏറ്റവും അടുത്ത വാക്കുകളുടെ സ്ഥാനങ്ങൾ എൻകോഡ് ചെയ്യുന്ന ലിസ്റ്റിന്റെ ആദ്യത്തെ `n` ഘടകങ്ങൾ എടുത്തു നൽകുന്നു.


In [105]:
def close_words(x,n=5):
  vec = embedder(vectorizer(x))[0]
  top5 = np.linalg.norm(vectors-vec,axis=1).argsort()[:n]
  return [ vocab[x] for x in top5 ]

close_words('paris')

['paris', 'philippines', 'seoul', 'jakarta', 'zoo']

In [112]:
close_words('china')

['china', 'russia', 'pakistan', 'israel', 'turkey']

In [113]:
close_words('official')

['official', 'military', 'office', 'police', 'sources']

## പ്രധാനപ്പെട്ട കാര്യങ്ങൾ

CBoW പോലുള്ള ബുദ്ധിമുട്ടുള്ള സാങ്കേതിക വിദ്യകൾ ഉപയോഗിച്ച്, Word2Vec മോഡൽ പരിശീലിപ്പിക്കാം. കേന്ദ്രവാക്ക് നൽകിയാൽ സമീപവാക്ക് പ്രവചിക്കാൻ പരിശീലിപ്പിക്കുന്ന skip-gram മോഡലും നിങ്ങൾ പരീക്ഷിച്ച് അതിന്റെ പ്രകടനം എത്രമാത്രം നല്ലതാണെന്ന് കാണാം.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
